In [1]:
import cmdstanpy as cmd
import pandas as pd
import numpy as np

from typing import Dict, Tuple, List, Optional

/home/braydentang/anaconda3/envs/cltv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Processing

In [2]:
raw_data = pd.read_table(
    "data/CDNOW_sample.txt",
    delim_whitespace=True,
    header=None, 
)

In [3]:
raw_data

,0,1,2,3,4
0,4,1,19970101,2,29.33
1,4,1,19970118,2,29.73
2,4,1,19970802,1,14.96
3,4,1,19971212,2,26.48
4,21,2,19970101,3,63.34
...,...,...,...,...,...
6914,23556,2356,19970726,3,45.74
6915,23556,2356,19970927,3,31.47
6916,23556,2356,19980103,2,28.98
6917,23556,2356,19980607,2,28.98


In [4]:
raw_data.columns = ["customer_id", "customer_index", "date", "quantity", "amount"]
new_data = raw_data.assign(
    new_date=lambda x: x["date"].astype("str"),
    date=lambda x: pd.to_datetime([f"{row[:4]}-{row[4:6]}-{row[6:]}" for row in x["new_date"]])
).drop(columns=["new_date"])

In [5]:
new_data

,customer_id,customer_index,date,quantity,amount
0,4,1,1997-01-01,2,29.33
1,4,1,1997-01-18,2,29.73
2,4,1,1997-08-02,1,14.96
3,4,1,1997-12-12,2,26.48
4,21,2,1997-01-01,3,63.34
...,...,...,...,...,...
6914,23556,2356,1997-07-26,3,45.74
6915,23556,2356,1997-09-27,3,31.47
6916,23556,2356,1998-01-03,2,28.98
6917,23556,2356,1998-06-07,2,28.98


In [6]:
train = new_data.query('date < "1998-01-01"')
test = new_data.query('date >= "1998-01-01"')

train

,customer_id,customer_index,date,quantity,amount
0,4,1,1997-01-01,2,29.33
1,4,1,1997-01-18,2,29.73
2,4,1,1997-08-02,1,14.96
3,4,1,1997-12-12,2,26.48
4,21,2,1997-01-01,3,63.34
...,...,...,...,...,...
6912,23556,2356,1997-06-10,2,26.73
6913,23556,2356,1997-07-19,2,29.33
6914,23556,2356,1997-07-26,3,45.74
6915,23556,2356,1997-09-27,3,31.47


In [7]:
train.loc[:, "amount_indicator"] = train.groupby("customer_id", as_index=False)["date"].transform(lambda x: x == x.min())
train_amount_filtered = train.loc[train["amount_indicator"] == 0, ["customer_id", "amount"]]
train_amount_filtered = train_amount_filtered.groupby("customer_id", as_index=False).agg("mean")
train_amount_filtered

/tmp/ipykernel_95138/1468150445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:, "amount_indicator"] = train.groupby("customer_id", as_index=False)["date"].transform(lambda x: x == x.min())


,customer_id,amount
0,4,23.723333
1,21,11.770000
2,111,75.347778
3,112,11.770000
4,114,25.550000
...,...,...
1035,23500,16.192500
1036,23506,13.970000
1037,23507,30.637500
1038,23551,44.928000


In [8]:
train_processed = train.groupby("customer_id", as_index=False).agg(
    frequency=("customer_id", "size"),
    recency=("date", lambda x: np.round((x.max() - x.min()).days / 7)),
    T=("date", lambda x: np.round((pd.to_datetime("1998-01-01") - x.min()).days / 7)),
).assign(
    frequency=lambda x: x["frequency"] - 1,
).merge(train_amount_filtered, on="customer_id")

In [9]:
train_processed

,customer_id,frequency,recency,T,amount
0,4,3,49.0,52.0,23.723333
1,21,1,2.0,52.0,11.770000
2,111,9,48.0,52.0,75.347778
3,112,1,5.0,52.0,11.770000
4,114,2,36.0,52.0,25.550000
...,...,...,...,...,...
1035,23500,8,28.0,40.0,16.192500
1036,23506,1,9.0,40.0,13.970000
1037,23507,4,38.0,40.0,30.637500
1038,23551,5,24.0,40.0,44.928000


In [10]:
customers_no_transactions_in_test = set(train["customer_id"]) - set(test["customer_id"])
customers_no_transactions_in_test

padded_test = pd.DataFrame(
    {
        "customer_id": [i for i in customers_no_transactions_in_test],
        "frequency": np.repeat(0, len(customers_no_transactions_in_test)),
        "monetary_value": np.repeat(0, len(customers_no_transactions_in_test)),
        "duration": np.round((test["date"].max() - pd.to_datetime("1998-01-01")).days / 7)
    }
)

In [11]:
test_processed = test.groupby("customer_id", as_index=False).agg(
    frequency=("customer_id", "size"),
    monetary_value=("amount", "mean")
).assign(
    duration=np.round((test["date"].max() - pd.to_datetime("1998-01-01")).days / 7),
)

test_processed = pd.concat([test_processed, padded_test])
test_processed.sort_values("customer_id")

,customer_id,frequency,monetary_value,duration
2,4,0,0.000,26.0
4,18,0,0.000,26.0
6,21,0,0.000,26.0
15,50,0,0.000,26.0
17,60,0,0.000,26.0
...,...,...,...,...
512,23537,2,19.775,26.0
1675,23551,0,0.000,26.0
513,23554,1,24.600,26.0
514,23556,2,28.980,26.0


## Model

Note: due to convergence issues with the original model (p ~ Beta(a, b)), switch to a logit-normal distribution for p instead. This isn't exactly the same thing as a Beta distribution, but it should be materially close enough.

This allows for a hierarchical parameterization that samples a lot better.

In [12]:
model = cmd.CmdStanModel(stan_file="stan/bg-nbd.stan")

In [13]:
data_dict = {
    "prior_only": 0,
    "N_customers": train_processed.shape[0],
    "recency": train_processed["recency"].astype(int).to_numpy(),
    "frequency": train_processed["frequency"].astype(int).to_numpy(),
    "T_age": train_processed["T"].astype(int).to_numpy()
}

samples = model.sample(
    data_dict,
    chains=4,
    seed=1234,
)

03:59:43 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status




chain 1 |▉         | 00:05 Iteration:    1 / 2000 [  0%]  (Warmup)
chain 1 |█▎        | 00:06 Iteration:  100 / 2000 [  5%]  (Warmup)


chain 1 |█▊        | 00:07 Iteration:  200 / 2000 [ 10%]  (Warmup)

chain 1 |██▎       | 00:07 Iteration:  300 / 2000 [ 15%]  (Warmup)


chain 1 |██▋       | 00:08 Iteration:  400 / 2000 [ 20%]  (Warmup)


chain 1 |███▏      | 00:09 Iteration:  500 / 2000 [ 25%]  (Warmup)


chain 1 |███▋      | 00:10 Iteration:  600 / 2000 [ 30%]  (Warmup)


chain 1 |████      | 00:11 Iteration:  700 / 2000 [ 35%]  (Warmup)


chain 1 |████▌     | 00:11 Iteration:  800 / 2000 [ 40%]  (Warmup)


chain 1 |█████     | 00:12 Iteration:  900 / 2000 [ 45%]  (Warmup)


chain 1 |█████▉    | 00:13 Iteration: 1001 / 2000 [ 50%]  (Sampling)


chain 1 |██████▎   | 00:14 Iteration: 1100 / 2000 [ 55%]  (Sampling)


chain 1 |██████▊   | 00:15 Iteration: 1200 / 2000 [ 60%]  (Sampling)


chain 1


04:00:07 - cmdstanpy - INFO - CmdStan done processing.
04:00:07 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: gamma_lpdf: Random variable[3] is 0, but must be positive finite! (in 'bg-nbd.stan', line 38, column 4 to column 44)
Exception: gamma_lpdf: Shape parameter is inf, but must be positive finite! (in 'bg-nbd.stan', line 38, column 4 to column 44)
	Exception: gamma_lpdf: Shape parameter is inf, but must be positive finite! (in 'bg-nbd.stan', line 38, column 4 to column 44)
	Exception: gamma_lpdf: Random variable[1] is 0, but must be positive finite! (in 'bg-nbd.stan', line 38, column 4 to column 44)
	Exception: gamma_lpdf: Random variable[7] is 0, but must be positive finite! (in 'bg-nbd.stan', line 38, column 4 to column 44)
Exception: gamma_lpdf: Random variable[1] is 0, but must be positive finite! (in 'bg-nbd.stan', line 38, column 4 to column 44)
	Exception: gamma_lpdf: Shape parameter is 0, but must be positive finite! (in 'bg-nbd.stan', line 38, column

## Ensuring Convergence

In [14]:
samples.diagnose()


'Processing csv files: /tmp/tmp9gn_54ey/bg-nbdkor3y4m2/bg-nbd-20240203035943_1.csv, /tmp/tmp9gn_54ey/bg-nbdkor3y4m2/bg-nbd-20240203035943_2.csv, /tmp/tmp9gn_54ey/bg-nbdkor3y4m2/bg-nbd-20240203035943_3.csv, /tmp/tmp9gn_54ey/bg-nbdkor3y4m2/bg-nbd-20240203035943_4.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\nNo divergent transitions found.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete, no problems detected.\n'

In [15]:
diagnostics = samples.summary()

In [16]:
diagnostics.sort_values("N_Eff", ascending=False)

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
z[57],0.064245,0.009483,0.996115,-1.57407,0.059208,1.70825,11034.100000,263.079000,0.999378
z[1001],0.033726,0.009813,1.024610,-1.65687,0.051874,1.74260,10901.900000,259.927000,0.999539
z[825],0.145989,0.009968,1.035880,-1.60509,0.174069,1.81030,10798.900000,257.472000,0.999496
z[475],0.137116,0.010198,1.053880,-1.62032,0.150856,1.85080,10679.200000,254.619000,0.999557
p_logit[57],-1.215350,0.019054,1.949100,-4.42733,-1.208010,1.99295,10463.700000,249.481000,0.999463
...,...,...,...,...,...,...,...,...,...
p_logit_mu,-1.342091,0.001988,0.109554,-1.51997,-1.339300,-1.16182,3038.099758,72.435739,0.999831
gamma_beta,11.679629,0.018683,0.847577,10.32400,11.652100,13.08340,2058.175492,49.071944,1.000179
gamma_alpha,1.598303,0.002206,0.095392,1.44560,1.594340,1.75854,1870.065325,44.586937,1.000038
p_logit_sigma,1.945360,0.005725,0.190226,1.64677,1.937180,2.26985,1103.896741,26.319602,1.002113


In [17]:
samples_array = samples.draws(concat_chains=True)
samples_array

array([[-1.22981e+04,  8.88792e-01,  1.61894e-01, ..., -3.40375e+00,
        -1.56551e+00, -8.85521e-01],
       [-1.22849e+04,  9.96003e-01,  1.61894e-01, ...,  4.89504e-01,
        -3.00940e+00, -3.84033e+00],
       [-1.23388e+04,  8.10114e-01,  1.61894e-01, ..., -7.13591e+00,
        -1.21974e+00, -2.88571e+00],
       ...,
       [-1.22486e+04,  9.37575e-01,  1.56071e-01, ..., -1.59329e+00,
        -1.92826e+00, -1.60473e+00],
       [-1.23073e+04,  8.74365e-01,  1.56071e-01, ..., -3.05857e+00,
        -3.36235e+00, -2.50180e+00],
       [-1.23252e+04,  9.41412e-01,  1.56071e-01, ..., -2.15993e+00,
        -1.97552e+00, -2.03684e-01]])

In [18]:
samples.column_names

('lp__',
 'accept_stat__',
 'stepsize__',
 'treedepth__',
 'n_leapfrog__',
 'divergent__',
 'energy__',
 'lambda[1]',
 'lambda[2]',
 'lambda[3]',
 'lambda[4]',
 'lambda[5]',
 'lambda[6]',
 'lambda[7]',
 'lambda[8]',
 'lambda[9]',
 'lambda[10]',
 'lambda[11]',
 'lambda[12]',
 'lambda[13]',
 'lambda[14]',
 'lambda[15]',
 'lambda[16]',
 'lambda[17]',
 'lambda[18]',
 'lambda[19]',
 'lambda[20]',
 'lambda[21]',
 'lambda[22]',
 'lambda[23]',
 'lambda[24]',
 'lambda[25]',
 'lambda[26]',
 'lambda[27]',
 'lambda[28]',
 'lambda[29]',
 'lambda[30]',
 'lambda[31]',
 'lambda[32]',
 'lambda[33]',
 'lambda[34]',
 'lambda[35]',
 'lambda[36]',
 'lambda[37]',
 'lambda[38]',
 'lambda[39]',
 'lambda[40]',
 'lambda[41]',
 'lambda[42]',
 'lambda[43]',
 'lambda[44]',
 'lambda[45]',
 'lambda[46]',
 'lambda[47]',
 'lambda[48]',
 'lambda[49]',
 'lambda[50]',
 'lambda[51]',
 'lambda[52]',
 'lambda[53]',
 'lambda[54]',
 'lambda[55]',
 'lambda[56]',
 'lambda[57]',
 'lambda[58]',
 'lambda[59]',
 'lambda[60]',
 'lam

# Predictive Checking

In [19]:
customer_index_lookup = {
    customer_id: customer_index + 1
    for customer_id, customer_index in 
    zip(train_processed["customer_id"], range(0, train_processed.shape[0]), strict=True)
}

In [20]:
def get_parameters_for_customer(
    customer_id: int,
    customer_index_lookup: Dict[int, int],
    samples: np.ndarray,
    samples_column_names: Tuple[str]
    ) -> Dict[str, np.ndarray]:
    id_lookup = f"[{customer_index_lookup[customer_id]}]"
    p_array_position = [idx for idx, name in enumerate(samples_column_names) if f"p{id_lookup}" in name]
    lambda_array_position = [idx for idx, name in enumerate(samples_column_names) if f"lambda{id_lookup}" in name]
    return {
        "p": samples[:, p_array_position].flatten(),
        "lambda": samples[:, lambda_array_position].flatten()
    }


test = [
    get_parameters_for_customer(idx, customer_index_lookup, samples_array, samples.column_names) 
    for idx in train_processed["customer_id"]
]

In [21]:
rng = np.random.default_rng(seed=1234)
def predictive_check(params: Dict[str, float], random_state: np.random.RandomState, age: Optional[int] = None) -> List[Dict[str, float]]:
    simulations = []
    for p, lamb in zip(params["p"], params["lambda"]):
        arrival_time_sims = []
        time = random_state.exponential(1/lamb)
        alive = True
        while alive:
            arrival_time_sims.append(time)
            next_arrival_time = random_state.exponential(1/lamb)
            time += next_arrival_time
            alive = random_state.uniform(0, 1) >= p
        if age is not None:
            simulations.append([time for time in arrival_time_sims if time <= age])
        else:
            simulations.append(arrival_time_sims)

    return simulations


In [22]:
test[0]

{'p': array([0.0287018, 0.108323 , 0.0903756, ..., 0.0139975, 0.366594 ,
        0.0313923]),
 'lambda': array([0.0445309, 0.0895281, 0.0444462, ..., 0.0451903, 0.125751 ,
        0.05117  ])}

In [23]:
predictive_check(params=test[0], random_state=rng, age=52)[0]

[34.21736915921333, 50.26579404256003]